# Monitoring Gradient Deployments with Arize AI

## Python Package Installs

In [24]:
!pip install -U xgboost -q

In [25]:
!pip install --upgrade twisted -q

In [26]:
!pip install attrs==19.2.0 -i http://mirrors.aliyun.com/pypi/simple --trusted-host mirrors.aliyun.com -q

In [38]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)

## Data Preprocessing

In [39]:
df = pd.read_csv('data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

target = ['Churn']
categorical_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod']
continuous_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

df.drop(df[df['TotalCharges'].str.strip() == ''].index, inplace=True)
df.reset_index(inplace=True, drop=True)
orig = df.copy()

df = pd.get_dummies(df.drop(['customerID'], axis=1), columns = categorical_cols)
df['Churn'] = np.where(df['Churn'] == 'Yes', 1, 0)

def min_max_normalize(col):
    return col / col.abs().max()

for col in continuous_cols:
    df[col] = df[col].astype('float64')
    df[col] = min_max_normalize(df[col])

## Train Test Split

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(['Churn'],axis=1), df['Churn'], test_size=0.25, random_state=17)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=17)

## Building and Saving XGBoost Classifier

In [41]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Create and train a XGBoost Classifier
xgb_cl = xgb.XGBClassifier(use_label_encoder=False)
xgb_cl.fit(X_train, y_train)

# Predict on test set
preds = xgb_cl.predict(X_test)
print(accuracy_score(y_test, preds))

0.7810011376564278


In [42]:
# Save model as .json file
xgb_cl.save_model('./models/xgb_cl_model.json')

## Preparing Training Dataset to be Logged into Arize

In [43]:
train_orig = orig[orig.index.isin(X_train.index)]
train_orig.reset_index(drop=True, inplace=True)

train_preds = xgb_cl.predict(X_train)

train_pred = pd.DataFrame(train_preds, columns=['Prediction'])
train_pred['Predicted_Churn'] = 'No'
train_pred.loc[train_pred['Prediction'] == 1, 'Predicted_Churn'] = 'Yes'
train_pred.drop(['Prediction'], axis=1, inplace=True)

combined_train_df = pd.concat([train_orig.reset_index(drop=True), train_pred], axis=1)

feature_cols = combined_train_df.drop(['customerID', 'Churn', 'Predicted_Churn'], axis=1).columns

## Integrating Arize API

In [44]:
!pip install arize -q

In [45]:
from arize.pandas.logger import Client, Schema
from arize.utils.types import Environments, ModelTypes

SPACE_KEY = "YOUR-SPACE-KEY"
API_KEY = "YOUR-API-KEY"

arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)

model_id = (
    "telco-churn-demo-model"  # This is the model name that will show up in Arize
)
model_version = "v1.0"  # Version of model - can be any string

if SPACE_KEY == "YOUR-SPACE-KEY" or API_KEY == "YOUR-API-KEY":
    raise ValueError("❌ NEED TO CHANGE SPACE AND/OR API_KEY")
else:
    print("✅ Arize setup complete!")

✅ Arize setup complete!


In [46]:
# Define a Schema() object for Arize to pick up data from the correct columns for logging
training_schema = Schema(
    prediction_id_column_name="customerID",
    prediction_label_column_name="Predicted_Churn",
    actual_label_column_name="Churn",
    feature_column_names=feature_cols,
)

# Logging Training DataFrame
training_response = arize_client.log(
    dataframe=combined_train_df,
    model_id=model_id,
    model_version=model_version,
    model_type=ModelTypes.SCORE_CATEGORICAL,
    environment=Environments.TRAINING,
    schema=training_schema,
)

# If successful, the server will return a status_code of 200
if training_response.status_code != 200:
    print(
        f"logging failed with response code {training_response.status_code}, {training_response.text}"
    )
else:
    print(f"✅ You have successfully logged training set to Arize")

Success! Check out your data at https://app.arize.com/organizations/QWNjb3VudE9yZ2FuaXphdGlvbjo0Mjg=/spaces/U3BhY2U6NDUx/models/modelName/telco-churn-demo-model?selectedTab=dataIngestion
✅ You have successfully logged training set to Arize
